In [1]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import json
from functools import partial

In [2]:
load_dotenv()

genai.configure(api_key=os.environ.get('GEMINI_API_KEY'))

## Model

We build a class to handle API calls to the LLM, in case we decide to change the LLM model.

Please make sure you have obtained the appropriate API key before continuing.

In [3]:
class AbstractModel:
    def __init__(self, base: str):
        pass

    def query(self, prompt: str) -> str:
        raise NotImplementedError

class QueryException(Exception):
    def __init__(self, message: str):
        super().__init__(f"Failed to query to the LLM: {message}")

In [4]:
class GeminiModel(AbstractModel):
    def __init__(self, base: partial[str]):
        self.base = base
        self.client = genai.GenerativeModel('gemini-1.5-pro')

    def query(self, prompt: str) -> str:
        response = self.client.generate_content(self.base(prompt=prompt)).candidates[0]
        if not hasattr(response, 'content'):
            raise QueryException('LLM response does not have content')
        return response.content.parts[0].text

## Prompt

We design the prompts and attempt to get the answer here.

### Category

The following prompt is designed to get the suitable category of product.

In [5]:
CATEGORY_PROMPT = """
<instruction>
You are a sales assistant on a large e-commerce platform.
Your job is to recommend the correct product to the customers.

The customer is finding a suitable gift.
The specific description of what the user wants is given in the "userquery" tag below.
You are to determine an appropriate category of product that the user can buy as gift.

The category you select should be listed in the "categories" tag below.
You must start your response with one of the categories below.
You are not to format the category in any way.
Failure to start your response with one of the categories below will lead to catastrophic effects.

You are then to explain your choice of category on the next line.
</instruction>

<categories>
Hobbies & Books
Books
Music, Movies & Games
Music Instruments
Figures & Model Kits
Electronics
Cameras
Fashion
Toys
Collectibles
</categories>

<userquery>
{prompt}
</userquery>
"""

In [6]:
gemini_category_model = GeminiModel(CATEGORY_PROMPT.format)

In [7]:
gemini_category_model.query("A gift for my friend, who loves to read Harry Potter.")

'Books\nThis is the most fitting category as it directly relates to reading and includes the Harry Potter series. \n'

In [8]:
gemini_category_model.query("How are you today?")

"Hobbies & Books \nThis is an open-ended question and doesn't give us much to go on, so suggesting Hobbies & Books is a good start as it offers a wide variety of potential gifts.  We need more information from the customer. \n"

### Product suggestion

The following prompt is designed to rank the products.

In [9]:
with open("products-books-sample.json", "r", encoding="utf-8") as f:
    products = json.loads(f.read())

In [10]:
PRODUCT_PROMPT = """
<instruction>
You are a sales assistant on a large e-commerce platform.
Your job is to recommend the correct product to the customers.

The customer is finding a suitable gift.
The specific description of what the user wants is given in the "userquery" tag below.
You are to rank the products based on the appropriateness for the gift occassion.

You are to follow the format specified in the "format" tag.
Failure to follow the format will lead to catastrophic consequences.
At the end of your response, starting on a new line, you are to explain your ranking.
</instruction>

<format>
[id of product 1]
[id of product 2]
[id of product 3]
...
</format>

<products>
{products}
</products>

<userquery>
{prompt}
</userquery>
"""

In [12]:
gemini_product_model = GeminiModel(partial(PRODUCT_PROMPT.format, products=products))

In [13]:
gemini_product_model.query("A gift for my friend, who loves to read Harry Potter.")

'1213824\n1241423\n4892132\n3218412\n1241251\n\nThis customer is looking for a gift for their friend who loves Harry Potter. Therefore, the Harry Potter book is ranked first. The other books are all novels and could also be enjoyed by an avid reader. \n'